In [ ]:
from sklearn.datasets import load_files
import numpy as np

In [ ]:
reviews_train = load_files("../data/aclImdb/train/")

In [ ]:
text_train, y_train = reviews_train.data, reviews_train.target

In [ ]:
type(text_train)

In [ ]:
len(text_train)

In [ ]:
text_train[6]

In [ ]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [ ]:
text_train[6]

In [ ]:
np.bincount(y_train)

##### Now We Load The Test Dataset

In [ ]:
reviews_test = load_files("../data/aclImdb/test/")

In [ ]:
text_test, y_test = reviews_test.data, reviews_test.target

In [ ]:
len(text_test)

In [ ]:
np.bincount(y_test)

In [ ]:
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]